# Localización de Hubs con Capacidad y Asignación Única (HLP)

---

## Introducción

El **Problema de Localización de Hubs con Capacidad y Asignación Única** (HLP, por sus siglas en inglés) es un problema de optimización combinatoria que se presenta en redes de transporte, telecomunicaciones y logística. En este problema, se busca determinar la ubicación óptima de hubs (centros de conexión) y la asignación de nodos a estos hubs, minimizando los costos totales de transporte.

### Enunciado del problema

Dado un conjunto de nodos (orígenes y destinos) y posibles ubicaciones para hubs, el objetivo es determinar qué hubs abrir y cómo asignar los nodos a estos hubs, minimizando el coste total de transporte entre los nodos a través de los hubs. En este caso, los hubs tienen capacidad limitada, y un nodo puede estar asignado a un solo hub.

### Variables de decisión

- $x_{ijhk}$: Variable continua que representa el flujo entre el nodo \(i\) y el nodo \(j\), pasando por los hubs \(h\) y \(k\).
- $y_h$: Variable binaria que toma el valor 1 si se abre un hub en la ubicación \(h\), y 0 en caso contrario.

### Parámetros

- $c_{ih}$: Coste de transporte entre el nodo \(i\) y el hub \(h\).
- $c_{hk}$: Coste de transporte entre los hubs \(h\) y \(k\).
- $c_{kj}$: Coste de transporte entre el hub \(k\) y el nodo \(j\).
- $f_h$: Coste fijo por abrir un hub en la ubicación \(h\).
- $w_{ij}$: Demanda de flujo entre los nodos \(i\) y \(j\).

### Función objetivo

El objetivo es minimizar el coste total de transporte y apertura de hubs, considerando los costos de transporte ajustados:

$$
\text{Minimizar} \quad \sum_{i} \sum_{j} \sum_{h} \sum_{k} w_{ij} \cdot C_{ijhk} \cdot x_{ijhk} + \sum_{h} f_h \cdot y_h
$$

donde $C_{ijhk}$ es el costo ajustado de transporte entre los nodos \(i\) y \(j\) pasando por los hubs \(h\) y \(k\), calculado considerando los factores de recolección, descuento y distribución.

### Restricciones

1. **Flujo entre nodos debe ser asignado a hubs**:

   Para cada par de nodos \(i, j\), el flujo debe ser completamente asignado a un par de hubs:

   $$
   \sum_{h} \sum_{k} x_{ijhk} = 1 \quad \forall i, \forall j
   $$

2. **Uso de hubs abiertos**:

   Un nodo solo puede asignar flujo a un hub si este está abierto:

   $$
   \sum_{k} x_{ijhk} \leq y_h \quad \forall i, \forall j, \forall h
   $$

3. **Número de hubs abiertos**:

   El número total de hubs abiertos no puede exceder el límite \(p\):

   $$
   \sum_{h} y_h \leq p
   $$

4. **Variables binarias**:

   $$
   x_{ijhk}, y_h \in \{0, 1\}
   $$

---

## Generación de Instancias Aleatorias

Para resolver el problema, se generarán instancias aleatorias que incluyan:

1. **Coordenadas**:
   - Coordenadas aleatorias en un plano bidimensional para representar la ubicación de los nodos y posibles hubs.

2. **Costos de Transporte**:
   - Calculados como la distancia euclidiana entre nodos y hubs, y ajustados mediante los factores de recolección, descuento y distribución.

3. **Demandas**:
   - Demandas aleatorias de flujo entre pares de nodos.

4. **Costos de Apertura**:
   - Valores aleatorios asociados a cada hub.

---

## Resolución del problema

A continuación, se presenta una implementación de este problema utilizando el paquete JuMP en Julia y el optimizador GLPK. El modelo busca minimizar el costo total de apertura de hubs y transporte, considerando tanto los costos de establecer hubs como los costos de transporte ajustados entre los nodos.


In [17]:
using JuMP, GLPK, Random

# Establecer semilla para reproducibilidad de los resultados
Random.seed!(12344213)

# Parámetros iniciales
n = 10  # Número de nodos
p = 4   # Número de hubs a abrir
V = 1:n # Conjunto de nodos
w = rand(1:10, n, n) # Demanda aleatoria entre nodos
f = rand(1:10, n)    # Costos para establecer un hub
c = rand(1:10, n, n) # Costos de transporte entre nodos

# Se asegura que los costos de transporte y demanda entre un nodo y sí mismo sean cero
for i in V
    c[i,i] = 0
    w[i,i] = 0
end

# Asegura que los costos de transporte cumplan con la desigualdad triangular
for i in V
    for j in V
        for k in V
            c[i,j] = min(c[i,j], c[i,k] + c[k,j])  # Se toma el mínimo entre el costo directo y el costo indirecto
        end
    end
end

# Factores de costo
α = 0.8 # Factor de descuento
χ = 1.2 # Factor de recolección
δ = 1.5 # Factor de distribución

# Calcula los costos de transporte ajustados considerando los factores anteriores
C = zeros(n, n, n, n)
for i in V, j in V, k in V, m in V
    C[i,j,k,m] = χ * c[i,k] + α * c[k,m] + δ * c[m,j]  # Ajuste de costos
end

# Función para resolver el problema de localización de hubs
function solveHLP()
    model = Model(GLPK.Optimizer)  # Crear el modelo con el optimizador GLPK

    # Variables del modelo
    @variable(model, y[k in V], Bin)  # Variable binaria para saber si se abre un hub en el nodo k
    @variable(model, X[i in V, j in V, k in V, m in V] >= 0)  # Variable para el flujo entre nodos i, j pasando por los hubs k, m

    # Objetivo: Minimizar el costo total de establecer hubs y el costo de transporte
    @objective(model, Min, sum(w[i, j] * C[i, j, k, m] * X[i, j, k, m] for i in V, j in V, k in V, m in V) + sum(f[k] * y[k] for k in V))

    # Restricciones
    @constraint(model, [i in V, j in V], sum(X[i, j, k, m] for k in V, m in V) == 1)  # Cada par de nodos debe tener un flujo
    @constraint(model, [i in V, j in V, k in V, m in V], X[i, j, k, m] <= y[k])  # El flujo solo puede ocurrir si el hub k está abierto
    @constraint(model, [i in V, j in V, k in V, m in V], X[i, j, k, m] <= y[m])  # El flujo solo puede ocurrir si el hub m está abierto
    @constraint(model, sum(y[k] for k in V) <= p)  # Número total de hubs abiertos no puede exceder p

    # Resolver el modelo
    optimize!(model)

    # Mostrar soluciones
    if termination_status(model) == OPTIMAL
        println("\nSolución óptima encontrada:")
        println("Costo total: ", objective_value(model))

        # Mostrar los hubs que están abiertos
        println("Hubs abiertos:")
        for k in V
            if value(y[k]) > 0.5 # Considerando y[k] como variable binaria
                println("- Hub $k")
            end
        end

        # Mostrar las asignaciones de flujo entre nodos
        println("\nAsignaciones de flujo:")
        for i in V, j in V
            if i != j
                for k in V, m in V
                    if value(X[i,j,k,m]) > 0  # Si hay flujo entre i y j pasando por k
                        println("Flujo entre nodo $i y nodo $j pasa por hub $k")
                    end
                end
            end
        end
    else
        println("infeasible")  # Si no se encuentra una solución óptima
    end 
end

# Llamada a la función para resolver el problema
solveHLP()



Solución óptima encontrada:
Costo total: 1965.3999999999996
Hubs abiertos:
- Hub 3
- Hub 5
- Hub 9
- Hub 10

Asignaciones de flujo:
Flujo entre nodo 1 y nodo 2 pasa por hub 5
Flujo entre nodo 1 y nodo 3 pasa por hub 9
Flujo entre nodo 1 y nodo 4 pasa por hub 5
Flujo entre nodo 1 y nodo 5 pasa por hub 5
Flujo entre nodo 1 y nodo 6 pasa por hub 5
Flujo entre nodo 1 y nodo 7 pasa por hub 5
Flujo entre nodo 1 y nodo 8 pasa por hub 5
Flujo entre nodo 1 y nodo 9 pasa por hub 9
Flujo entre nodo 1 y nodo 10 pasa por hub 5
Flujo entre nodo 2 y nodo 1 pasa por hub 5
Flujo entre nodo 2 y nodo 3 pasa por hub 5
Flujo entre nodo 2 y nodo 4 pasa por hub 5
Flujo entre nodo 2 y nodo 5 pasa por hub 5
Flujo entre nodo 2 y nodo 6 pasa por hub 5
Flujo entre nodo 2 y nodo 7 pasa por hub 5
Flujo entre nodo 2 y nodo 8 pasa por hub 5
Flujo entre nodo 2 y nodo 9 pasa por hub 9
Flujo entre nodo 2 y nodo 10 pasa por hub 5
Flujo entre nodo 3 y nodo 1 pasa por hub 3
Flujo entre nodo 3 y nodo 2 pasa por hub 3
Flujo